In [1]:
from server import LinkedViewsServer
data_folder = "/scratch/visual/bzfharth/in-silico-install-dir/project_src/in_silico_framework/getting_started/linked-views-example-data/backend_data_2023-06-22"

server = LinkedViewsServer(data_folder)
server.start(5000)
#server.stop()

start read: simulation_samples_original.csv
complete read
start read: simulation_samples.csv
complete read
server is running at port 5000
logs are written to /tmp/linked-views-server.log
set data:        server.set_data(vaex_df)
stop server:     server.stop()


In [2]:
import vaex
import vaex.ml
import util

def append_PCA(df, columns, descriptor, n_components = 2):
    pca = vaex.ml.PCA(features=columns, n_components=n_components)    
    df_transformed = pca.fit_transform(df)
    for component_idx in range(0, n_components):
        df_transformed.rename(f"PCA_{component_idx}", f"{descriptor}-{component_idx+1}")
    return df_transformed

In [3]:
filename = "/scratch/visual/bzfharth/in-silico-install-dir/project_src/in_silico_framework/getting_started/linked-views-example-data/backend_data_2023-06-22/simulation_samples.csv"
df = vaex.from_csv(filename, copy_index=False)

df_extended = append_PCA(df, util.getParamsCols(), "PCA-ephys")

server.set_data(df_extended)


In [4]:
if("global" in server.selections):
    columns = server.selections["global"]["columns"] # ['ephys.CaDynamics_E2_v2.apic.decay','ephys.NaTa_t.axon.gNaTa_tbar']
    col1 = columns[0]
    col2 = columns[1]

    print(columns)

    ranges = server.selections["global"]["bin_ranges"]

    server.vaex_df.select_nothing(name="selection_global")
    for idx, range_i in enumerate(ranges):    
        limit_i = [(range_i[0][0], range_i[0][1]), (range_i[1][0], range_i[1][1])]        
        server.vaex_df.select_rectangle(server.vaex_df[col1], server.vaex_df[col2], limit_i, name="selection_global", mode="or")
        #print(limit_i)

    df_selected_indices = server.vaex_df.evaluate(server.vaex_df["Unnamed: 0"], selection="selection_global")

    print("num selected", df_selected_indices.shape)
    print("indices", df_selected_indices)